## This is a repeat for Ansar's Mie code, which is used for understanding the Mie Scattering

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Size Parameter: x
# Index of Refraction: m
# y = x*m

In [2]:
## Riccati-Bessel Function Calculations     
def A(k, y):
    if k == 0:
        return np.cos(y)/np.sin(y)
    else:
        return -k/y + (k/y - A(k-1,y))**(-1)
    
def L(k, x):
    if k == -1:
        return np.cos(x) + complex(0,1)*np.sin(x)
    if k == 0:
        return np.sin(x) - complex(0,1)*np.cos(x)
    else:
        return (2*k-1)/x*L(k-1, x) - L(k-2, x)
    
def a(k, x, m):
    y = x * m
    return ((A(k,y)/m + k/x)*np.real(L(k, x)) - np.real(L(k-1, x)))/((A(k,y)/m + k/x)*L(k, x) - L(k-1, x))
    
def b(k, x, m):
    y = x * m
    return ((A(k,y)*m + k/x)*np.real(L(k, x)) - np.real(L(k-1, x)))/((A(k,y)*m + k/x)*L(k, x) - L(k-1, x))
  
def pi(k, theta):
    if k == 0:
        return 0
    if k == 1:
        return 1
    else:
        return (2*k - 1)*np.cos(theta)*pi(k - 1, theta)/(k - 1) - k*pi(k - 2, theta)/(k - 1)
    
def t(k, theta):
    return k*np.cos(theta)*pi(k, theta) - (k + 1)*pi(k - 1, theta); 

In [3]:
#### Cross Section Calculations ####
# Summation definitions
def rieSum1(x, m):
    sum = 0
    for k in range(1,20):
        sum += (2*k + 1)*(np.real(a(k, x, m))**2 + np.imag(a(k, x, m))**2 + np.real(b(k, x, m))**2 + np.imag(b(k, x, m))**2)
    return sum

def rieSum2(x, m):
    sum = 0
    for k in range(1,20):
        sum += (2*k + 1)*np.real(a(k, x, m) + b(k, x, m))
    return sum

def rieSum3(x, m):
    sum = 0
    for k in range(1,20):
        sum += (2*k + 1)*(-1)**k*(a(k, x, m) - b(k, x, m))
    return sum

def S1(theta, x, m):
    sum = 0
    for k in range(1,20):
        sum += (2*k + 1)/(k*(k + 1))*(a(k, x, m)*pi(k, theta) + b(k, x, m)*t(k, theta))
    return sum

def S2(theta, x, m):
    sum = 0
    for k in range(1,20):
        sum += (2*k + 1)/(k*(k + 1))*(b(k, x, m)*pi(k, theta) + a(k, x, m)*t(k, theta))
    return sum

In [4]:
# Functions for evaluating cross sections
def scatter_eff(x, m):
    return (2/x**2)*rieSum1(x, m)

def extinct_eff(x, m):
    return (2/x**2)*rieSum2(x, m)

def backscatter_eff(x, m):
    return (1/x**2) * np.abs(rieSum3(x, m))**2


# Phase function
def P(theta, x, m):
    return (np.abs(S1(theta, x, m))**2 + np.abs(S2(theta, x, m))**2)/(rieSum1(x, m))


In [20]:
wavelength = 2 # [um]
diameter = 0.0695 # [um]
m = complex(1.384, 1.260E-03)
x = np.pi*diameter/wavelength
print(f"Liquid drop scattering efficiency: {scatter_eff(x, m):.5f}" )
print(f"Liquid drop extinction efficiency: {extinct_eff(x, m):.5f}" )
print(f"Liquid drop back scattering efficiency: {backscatter_eff(x, m):.5f}" )
print(f"Liquid drop back scattering cross section: {(np.pi*1**2/4)*backscatter_eff(x, m):.5f}" )


Liquid drop scattering efficiency: 0.00002
Liquid drop extinction efficiency: 0.00032
Liquid drop back scattering efficiency: 0.00003
Liquid drop back scattering cross section: 0.00002
